## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
from google.colab import files
uploaded = files.upload()

Saving charity_data.csv to charity_data.csv


In [2]:
application_df = pd.read_csv('charity_data.csv')
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
app_types = application_df['APPLICATION_TYPE'].value_counts()
app_types

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

#choosing types with values less than 500

application_types_to_replace = []

for index, value in app_types.items():
  if value < 500:
    application_types_to_replace.append(index)
    print(f"Index : {index}, Value : {value}")


application_types_to_replace

Index : T9, Value : 156
Index : T13, Value : 66
Index : T12, Value : 27
Index : T2, Value : 16
Index : T25, Value : 3
Index : T14, Value : 3
Index : T29, Value : 2
Index : T15, Value : 2
Index : T17, Value : 1


['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [7]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Look at CLASSIFICATION value counts for binning
class_types = application_df['CLASSIFICATION'].value_counts()
class_types

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [9]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for index, value in class_types.items():
  if value > 1:
    print(f"Index : {index}, Value : {value}")

Index : C1000, Value : 17326
Index : C2000, Value : 6074
Index : C1200, Value : 4837
Index : C3000, Value : 1918
Index : C2100, Value : 1883
Index : C7000, Value : 777
Index : C1700, Value : 287
Index : C4000, Value : 194
Index : C5000, Value : 116
Index : C1270, Value : 114
Index : C2700, Value : 104
Index : C2800, Value : 95
Index : C7100, Value : 75
Index : C1300, Value : 58
Index : C1280, Value : 50
Index : C1230, Value : 36
Index : C1400, Value : 34
Index : C7200, Value : 32
Index : C2300, Value : 32
Index : C1240, Value : 30
Index : C8000, Value : 20
Index : C7120, Value : 18
Index : C1500, Value : 16
Index : C1800, Value : 15
Index : C6000, Value : 15
Index : C1250, Value : 14
Index : C8200, Value : 11
Index : C1238, Value : 10
Index : C1278, Value : 10
Index : C1235, Value : 9
Index : C1237, Value : 9
Index : C7210, Value : 7
Index : C2400, Value : 6
Index : C1720, Value : 6
Index : C4100, Value : 6
Index : C1257, Value : 5
Index : C1600, Value : 5
Index : C1260, Value : 3
Inde

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# binning everything with less than 100 rows
classifications_to_replace = []
for index, value in class_types.items():
  if value < 100:
    classifications_to_replace.append(index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
types = application_df.dtypes
for index, value in types.items():
  if value == 'object':
    print(index)
    ohe = pd.get_dummies(application_df[index])
    application_df = pd.merge(application_df, ohe, left_index=True, right_index=True)
    application_df = application_df.drop(columns=[index])

APPLICATION_TYPE
AFFILIATION
CLASSIFICATION
USE_CASE
ORGANIZATION
INCOME_AMT
SPECIAL_CONSIDERATIONS


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Other_x'} in the result is deprecated and will raise a MergeError in a future version.
  import sys


In [12]:
application_df.dtypes

STATUS              int64
ASK_AMT             int64
IS_SUCCESSFUL       int64
Other_x             uint8
T10                 uint8
T19                 uint8
T3                  uint8
T4                  uint8
T5                  uint8
T6                  uint8
T7                  uint8
T8                  uint8
CompanySponsored    uint8
Family/Parent       uint8
Independent         uint8
National            uint8
Other_y             uint8
Regional            uint8
C1000               uint8
C1200               uint8
C1270               uint8
C1700               uint8
C2000               uint8
C2100               uint8
C2700               uint8
C3000               uint8
C4000               uint8
C5000               uint8
C7000               uint8
Other_x             uint8
CommunityServ       uint8
Heathcare           uint8
Other_y             uint8
Preservation        uint8
ProductDev          uint8
Association         uint8
Co-operative        uint8
Corporation         uint8
Trust       

In [13]:
# Split our preprocessed data into our features and target arrays
target = application_df['IS_SUCCESSFUL']
data = application_df.drop(columns=['IS_SUCCESSFUL'])
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputs = X_train_scaled.shape[1]
inputs

49

In [16]:

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation="relu",input_dim=inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=150, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 200)               10000     
                                                                 
 dense_1 (Dense)             (None, 150)               30150     
                                                                 
 dense_2 (Dense)             (None, 1)                 151       
                                                                 
Total params: 40,301
Trainable params: 40,301
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5341 - accuracy: 0.7405
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5319 - accuracy: 0.7400
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5306 - accuracy: 0.7407
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5314 - accuracy: 0.7402
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5335 - accuracy: 0.7399
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5323 - accuracy: 0.7396
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5311 - accuracy: 0.7402
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5308 - accuracy: 0.7404
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5303 - accuracy: 0.7400
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5308 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5982 - accuracy: 0.7326 - 553ms/epoch - 2ms/step
Loss: 0.5982071161270142, Accuracy: 0.7325947284698486


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

First run: one layer with 150 nodes  
268/268 - 0s - loss: 0.5620 - accuracy: 0.7318 - 364ms/epoch - 1ms/step  
Loss: 0.5619537234306335, Accuracy: 0.7317784428596497

Second run: two layers with 150 nodes  
268/268 - 1s - loss: 0.7621 - accuracy: 0.7308 - 526ms/epoch - 2ms/step  
Loss: 0.762144923210144, Accuracy: 0.7308454513549805

Third run: two layers, with 200, 150 nodes  
268/268 - 0s - loss: 0.9207 - accuracy: 0.7318 - 466ms/epoch - 2ms/step  
Loss: 0.9206628203392029, Accuracy: 0.7317784428596497